In [1]:
import hashlib as hl
import csv
from datetime import datetime
from os import close
import numpy as np
import logging
import argparse
import sys
from datetime import datetime
from datetime import timezone
import socket
import math
import pickle
import random
import pandas as pd
#from Functions.Functions_wireshark import *
from Functions.Optimized_Parser_Labelling import *
log = logging.getLogger()
log.setLevel(logging.DEBUG)

from Functions.Optimized_Parser_Labelling import pcap_parser

In [2]:
## Reading pre-processed CSV File of CICIDS

In [2]:
df_p=pd.read_csv('./CICIDS2017_preprocessed.csv')
df_p=df_p[['Timestamp','Source IP','Destination IP','Destination Port','Source Port','Protocol','Label']]
df_p.rename(columns={'Timestamp': 'stime', 'Source IP': 'srcip', 'Destination IP': 'dstip', 'Destination Port': 'dsport', 'Source Port': 'sport', 'Label': 'label','Protocol': 'protocol_m'}, inplace=True)

#Converting time into epoch format along with the mitigation of AM/PM issue.
df_p['stime']=df_p['stime'].apply(lambda x: (datetime.strptime(x, '%d/%m/%Y %H:%M')) if x.count(':') == 1 else  (datetime.strptime(x, '%d/%m/%Y %H:%M:%S')))
df_p['stime']=df_p['stime'].apply(lambda x: int((datetime(x.year,x.month,x.day,(x.hour+15),x.minute,x.second,tzinfo=timezone.utc)).timestamp()) if (x.hour>=1)&(x.hour<=7) else int((datetime(x.year,x.month,x.day,(x.hour+3),x.minute,x.second,tzinfo=timezone.utc)).timestamp()))
df_p=df_p.sort_values(by='stime')

In [3]:
df_p.columns

Index(['stime', 'srcip', 'dstip', 'dsport', 'sport', 'protocol_m', 'label'], dtype='object')

In [4]:
df_p['protocol_m'] = df_p['protocol_m'].astype(str)
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('6.0', 'tcp'))
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('17.0', 'udp'))
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('0.0', 'other'))

In [5]:
df_p.label.value_counts()

BENIGN                        2271122
DoS Hulk                       230123
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack – Brute Force         1507
Web Attack – XSS                  652
Infiltration                       36
Web Attack – Sql Injection         21
Heartbleed                         11
Name: label, dtype: int64

In [6]:
## Labeling & Combining of Parsed PCAP Files

In [7]:
def time_processing(combine, num):
    ##  DDoS LOIT (15:56 – 16:16)
    # combine.drop(combine[(combine.stime>=1499453760 )&(combine.stime<=1499454960 )&(combine.label=='BENIGN')].index,inplace=True)
    
    if num == 5:
        combine.drop(
            combine[(combine['stime'] >= 1499453760) & (combine['stime'] <= 1499454960) & (combine['label'] == 'BENIGN')].index,
            inplace=True)
        ## Port Scan (13:55-15:30)
        combine.drop(combine[(combine.stime >= 1499446500) & (combine.stime <= 1499452200) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## Botnet ARES (10:02 a.m. – 11:02 a.m.)
        combine.drop(combine[(combine.stime >= 1499432520) & (combine.stime <= 1499436120) & (combine.label == 'BENIGN')].index,
                     inplace=True)
    elif num == 4:
        ## Thursday
        ## Infiltration – Dropbox download (15:04 – 15:45 p.m.)
        combine.drop(combine[(combine.stime >= 1499364240) & (combine.stime <= 1499366700) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## Infiltration – Dropbox download and (14:33 -14:35)
        combine.drop(combine[(combine.stime >= 1499362380) & (combine.stime <= 1499362500) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## Web Attack – Sql Injection (10:40 – 10:42 a.m.)
        combine.drop(combine[(combine.stime >= 1499348400) & (combine.stime <= 1499348520) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## Web Attack – XSS (10:15 – 10:35 a.m.)
        combine.drop(combine[(combine.stime >= 1499346900) & (combine.stime <= 1499348100) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## Web Attack – Brute Force (9:20 – 10 a.m.)
        combine.drop(combine[(combine.stime >= 1499343600) & (combine.stime <= 1499346000) & (combine.label == 'BENIGN')].index,
                     inplace=True)
    
    elif num == 3:
        ## Wednesday
        ## Heartbleed Port 444 (15:12 - 15:32)
        combine.drop(combine[(combine.stime >= 1499278320) & (combine.stime <= 1499279520) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## DoS GoldenEye (11:10 – 11:23 a.m.)
        combine.drop(combine[(combine.stime >= 1499263800) & (combine.stime <= 1499264580) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## DoS Hulk (10:43 – 11 a.m.)
        combine.drop(combine[(combine.stime >= 1499262180) & (combine.stime <= 1499263200) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ##DoS Slowhttptest (10:14 – 10:35 a.m.)
        combine.drop(combine[(combine.stime >= 1499260440) & (combine.stime <= 1499261700) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ## DoS slowloris (9:47 – 10:10 a.m.)
        combine.drop(combine[(combine.stime >= 1499258820) & (combine.stime <= 1499260200) & (combine.label == 'BENIGN')].index,
                     inplace=True)
    
    elif num == 2:
    
        ## Tuesday
        ## FTP-Patator (9:20 – 10:20 a.m.) 
        combine.drop(combine[(combine.stime >= 1499170800) & (combine.stime <= 1499174400) & (combine.label == 'BENIGN')].index,
                     inplace=True)
        ##SSH-Patator (14:00 – 15:00 p.m.)
        combine.drop(combine[(combine.stime >= 1499187600) & (combine.stime <= 1499191200) & (combine.label == 'BENIGN')].index,
                     inplace=True)

In [8]:
# Monday

df = pd.read_csv('./CICIDS_results\pcap_file_csv_parser\pcap_csv_1.csv', index_col=0)
df = df.sort_values(by='stime')

stime = df.stime[0]
ltime = int(df.stime.tail(1))
print("Start Value:", df.stime[0])
print("End Value:", int(df.stime.tail(1)))

df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

aa = df_p[(df_p['stime'] >= stime) & (df_p['stime'] <= ltime)]

df.protocol_m.value_counts()
aa.protocol_m.value_counts()
combine = df.merge(aa, left_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'],
                   right_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'])

combine.drop_duplicates(inplace=True)
time_processing(combine, 1)

combine.to_csv('./CICIDS_results\Labelled_pcap_file\labelled_pcap_csv_1.csv', index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\steph\PycharmProjects\payloadbyte\.venv\lib\site-packages\pandas\io\parsers\c_parser_wrapper.py", line 224, in read
    chunks = self._reader.read_low_memory(nrows)
  File "pandas\_libs\parsers.pyx", line 801, in pandas._libs.parsers.TextReader.read_low_memory
  File "pandas\_libs\parsers.pyx", line 880, in pandas._libs.parsers.TextReader._read_rows
  File "pandas\_libs\parsers.pyx", line 1039, in pandas._libs.parsers.TextReader._convert_column_data
  File "C:\Users\steph\PycharmProjects\payloadbyte\.venv\lib\site-packages\pandas\core\dtypes\common.py", line 1465, in is_extension_array_dtype
    dtype = getattr(arr_or_dtype, "dtype", arr_or_dtype)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\steph\PycharmProjects\payloadbyte\.venv\lib\site-packages\pandas\io\parsers\readers.py", line 488, in _read
    return parser.read(nrows)
  File "C:\Use

INFO:root:
Unfortunately, your original traceback can not be constructed.



TypeError: object of type 'NoneType' has no len()

In [19]:
# Tuesday

df = pd.read_csv('./CICIDS_results\pcap_file_csv_parser\pcap_csv_2.csv', index_col=0)
df = df.sort_values(by='stime')
# df=df.sort_values(by='stime')
stime = df.stime[0]
ltime = int(df.stime.tail(1))
print("Start Value:", df.stime[0])
print("End Value:", int(df.stime.tail(1)))

df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

aa = df_p[(df_p['stime'] >= stime) & (df_p['stime'] <= ltime)]

df.protocol_m.value_counts()
aa.protocol_m.value_counts()
combine = df.merge(aa, left_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'],
                   right_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'])

combine.drop_duplicates(inplace=True)
time_processing(combine, 2)

combine.to_csv('./CICIDS_results\Labelled_pcap_file\labelled_pcap_csv_2.csv', index=False)

Start Value: 1499169224
End Value: 1499198431


In [20]:
df = pd.read_csv('./CICIDS_results\pcap_file_csv_parser\pcap_csv_3.csv', index_col=0)
df = df.sort_values(by='stime')

stime = df.stime[0]
ltime = int(df.stime.tail(1))
print("Start Value:", df.stime[0])
print("End Value:", int(df.stime.tail(1)))

df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

aa = df_p[(df_p['stime'] >= stime) & (df_p['stime'] <= ltime)]

df.protocol_m.value_counts()
aa.protocol_m.value_counts()

combine = df.merge(aa, left_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'],
                   right_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'])

combine.drop_duplicates(inplace=True)
time_processing(combine, 3)

combine.to_csv('./CICIDS_results\Labelled_pcap_file\labelled_pcap_csv_3.csv', index=False)

Start Value: 1499254962
End Value: 1499285415


In [21]:
df = pd.read_csv('./CICIDS_results\pcap_file_csv_parser\pcap_csv_4.csv', index_col=0)
df = df.sort_values(by='stime')

stime = df.stime[0]
ltime = int(df.stime.tail(1))
print("Start Value:", df.stime[0])
print("End Value:", int(df.stime.tail(1)))

df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

aa = df_p[(df_p['stime'] >= stime) & (df_p['stime'] <= ltime)]

df.protocol_m.value_counts()
aa.protocol_m.value_counts()
combine = df.merge(aa, left_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'],
                   right_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'])

combine.drop_duplicates(inplace=True)
time_processing(combine, 4)

combine.to_csv('./CICIDS_results\Labelled_pcap_file\labelled_pcap_csv_4.csv', index=False)

Start Value: 1499342340
End Value: 1499371484


In [22]:
df = pd.read_csv('./CICIDS_results\pcap_file_csv_parser\pcap_csv_5.csv', index_col=0)
df = df.sort_values(by='stime')

stime = df.stime[0]
ltime = int(df.stime.tail(1))
print("Start Value:", df.stime[0])
print("End Value:", int(df.stime.tail(1)))

df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

aa = df_p[(df_p['stime'] >= stime) & (df_p['stime'] <= ltime)]

df.protocol_m.value_counts()
aa.protocol_m.value_counts()
combine = df.merge(aa, left_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'],
                   right_on=['srcip', 'dstip', 'dsport', 'sport', 'protocol_m'])

combine.drop_duplicates(inplace=True)
time_processing(combine, 5)

combine.to_csv('./CICIDS_results\Labelled_pcap_file\labelled_pcap_csv_5.csv', index=False)

Start Value: 1499428790
End Value: 1499457761


In [9]:
def combine_CICIDS(in_file_path,out_path):
    combine=pd.DataFrame(columns=['srcip', 'sport', 'dstip', 'dsport', 'protocol_m',
     'sttl', 'total_len', 'payload', 't_delta', 'stime','label'])
    for files in in_file_path:
        df=pd.read_csv(files)
        combine=combine.append(df, ignore_index=True)
        print(combine.shape)
    csv_out=out_path+"combined_labelled_pcap_csv.csv"
    logging.info("Exporting_combined_csv_file....")
    combine.to_csv(csv_out,index=False)
    return combine

In [10]:
label_csv= './CICIDS_results\Labelled_pcap_file\\'
in_file=[]
for x in range(1,6): ## Starting & Ending values for the files
    in_file.append(label_csv+"labelled_pcap_csv_"+str(x)+".csv")

out_path= './CICIDS_results\Labelled_pcap_file\Combined\\'

In [11]:
df_payload=combine_CICIDS(in_file,out_path)

(9688673, 11)
(15286978, 11)
(32939547, 11)
(38343550, 11)
(43916974, 11)


INFO:root:Exporting_combined_csv_file....


In [22]:
df_payload = pd.read_csv('./CICIDS_results\Labelled_pcap_file\Combined/combined_labelled_pcap_csv.csv')

In [23]:
df_payload.shape

(43916974, 11)

In [24]:
df_payload.drop_duplicates(inplace=True)

In [25]:
df_payload.label.value_counts()

BENIGN                        31545181
DoS Hulk                       9211652
DDoS                           1495765
DoS GoldenEye                   489471
DoS slowloris                   375252
DoS Slowhttptest                312365
PortScan                        166146
Infiltration                    122169
SSH-Patator                      78740
FTP-Patator                      50141
Web Attack – Brute Force         27948
Heartbleed                       23320
Bot                              10422
Web Attack – XSS                  8335
Web Attack – Sql Injection          67
Name: label, dtype: int64

In [26]:
df_payload.drop(df_payload[df_payload.payload.isnull()].index,inplace=True)

In [27]:
df_payload.shape

(32063006, 11)

In [28]:
x=df_payload['payload']
new=[]
for p in range(len(x)):
    try:
        o=(int((x.iloc[p]), 16))
        if o>0:
            new.append(1)
        else:
            new.append(0)
    except Exception as e:
        print(p)
        new.append('Big')

In [29]:
df_payload['payload_int']=new

In [30]:
df_payload.drop(df_payload[df_payload.payload_int==0].index,inplace=True)

In [31]:
## After Removing zero payload
df_payload.label.value_counts()

BENIGN                        21807457
DoS Hulk                       2495632
DDoS                            241405
DoS GoldenEye                   128122
Infiltration                    122011
DoS slowloris                   121097
DoS Slowhttptest                 80542
SSH-Patator                      56774
FTP-Patator                      31843
Heartbleed                       13486
Web Attack – Brute Force         11754
Web Attack – XSS                  3341
Bot                               2543
PortScan                           830
Web Attack – Sql Injection          12
Name: label, dtype: int64

In [32]:
df_payload.pop('payload_int')

5           1
9           1
12          1
13          1
17          1
           ..
43916963    1
43916964    1
43916967    1
43916968    1
43916973    1
Name: payload_int, Length: 25116849, dtype: int64

In [33]:
df_payload.sttl=df_payload.sttl.astype('int32')
df_payload.dsport=df_payload.dsport.astype('int32')
df_payload.sport=df_payload.sport.astype('int32')
df_payload.total_len=df_payload.total_len.astype('int32')

In [34]:
df_payload.to_csv(r'./CICIDS_results\Labelled_pcap_file\Combined\combined_labelled_cleaned_sorted_pcap_csv.csv',index=False)

In [ ]:
df_payload= pd.read_csv(r'./CICIDS_results\Labelled_pcap_file\Combined\combined_labelled_cleaned_sorted_pcap_csv.csv')

In [35]:
df_payload.iloc[:,:-1]

,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,payload,t_delta,stime
5,192.168.10.9,1034,192.168.10.3,88,tcp,128,1437,000005716c82056d30820569a103020105a20302010ca3...,0.000002,1499082998
9,192.168.10.9,1034,192.168.10.3,88,tcp,128,1437,000005716c82056d30820569a103020105a20302010ca3...,0.000046,1499082998
12,192.168.10.3,88,192.168.10.9,1034,tcp,128,1450,0000057e6d82057a30820576a003020105a10302010da3...,0.000049,1499082998
13,192.168.10.3,88,192.168.10.9,1034,tcp,128,1450,0000057e6d82057a30820576a003020105a10302010da3...,0.000284,1499082998
17,192.168.10.9,1030,192.168.10.3,445,tcp,128,158,00000072fe534d42400001000000000003000100080000...,0.000422,1499082998
...,...,...,...,...,...,...,...,...,...,...
43916963,192.168.10.50,22,192.168.10.17,37759,tcp,64,104,3d1b3db4bab926f564f3a4d39d932a0b37a9602b4adc13...,0.000002,1499457720
43916964,192.168.10.50,22,192.168.10.17,37759,tcp,64,104,3d1b3db4bab926f564f3a4d39d932a0b37a9602b4adc13...,0.000001,1499457720
43916967,192.168.10.50,22,192.168.10.17,37759,tcp,64,900,0000033c081f00000117000000077373682d7273610000...,0.004842,1499457720
43916968,192.168.10.50,22,192.168.10.17,37759,tcp,64,900,0000033c081f00000117000000077373682d7273610000...,0.000004,1499457720


In [36]:
from imblearn.under_sampling import RandomUnderSampler 

In [37]:
dict1={ 'BENIGN': 362108,
'DoS Hulk':          250000,
'DDoS'  :         214200,
'DoS GoldenEye':     128122,
'DoS slowloris':    121097,
'Infiltration'        :103700,
'DoS Slowhttptest'         :  80542,
'SSH-Patator':          48165,
'FTP-Patator'            :   31843,
'Heartbleed'              :  13486,
'Web Attack – Brute Force'            :   11700,
'Web Attack – XSS'              :  3300,
'Bot'            :   2543,
'PortScan'              :  830,
'Web Attack – Sql Injection': 12
}

In [38]:
rus = RandomUnderSampler(random_state=42,sampling_strategy=dict1)

In [39]:
X_res, y_res = rus.fit_resample(df_payload.iloc[:,:-1], df_payload.iloc[:,-1])

In [40]:
X_res['label']=y_res

In [2]:
X_res.to_csv(r'./CICIDS_results\Labelled_pcap_file\Combined\combined_labelled_cleaned_undersampled_pcap_csv.csv',index=False)

NameError: name 'X_res' is not defined

In [9]:
X_res = pd.read_csv('./CICIDS_results/Labelled_pcap_file/Combined/combined_labelled_cleaned_undersampled_pcap_csv.csv')
y_res = X_res['label']

In [4]:
X_res.iloc[:,:-1]

,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,payload,t_delta,stime
0,205.185.208.27,443,192.168.10.15,60233,tcp,57,2960,17030310186261c249c0ba72ce9dc971d07711c1c371b1...,0.000319,1499367000
1,13.107.4.50,80,192.168.10.15,49672,tcp,118,2960,f9806d4baa3b99af2b0426787219da99656887dd70d68e...,0.000117,1499256360
2,192.168.10.3,61378,192.168.10.1,53,udp,128,83,d6d7010000010000000000010361647306616461707476...,0.000250,1499105296
3,192.168.10.3,60342,192.168.10.1,53,udp,128,76,4ede0100000100000000000107737570706f7274076d6f...,0.000439,1499359560
4,192.168.10.16,137,192.168.10.255,137,udp,64,78,35c3011000010000000000002046454546464446454543...,0.000105,1499178240
...,...,...,...,...,...,...,...,...,...,...
1371643,172.16.0.1,57684,192.168.10.50,80,tcp,62,435,474554202f64762f76756c6e65726162696c6974696573...,0.160490,1499347440
1371644,172.16.0.1,59042,192.168.10.50,80,tcp,62,637,474554202f64762f76756c6e65726162696c6974696573...,0.000003,1499347560
1371645,172.16.0.1,59042,192.168.10.50,80,tcp,62,435,474554202f64762f76756c6e65726162696c6974696573...,0.235294,1499347560
1371646,172.16.0.1,56994,192.168.10.50,80,tcp,62,637,474554202f64762f76756c6e65726162696c6974696573...,0.030888,1499347440


In [5]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.preprocessing import LabelEncoder
from Functions.Pipeline import *

In [6]:
## Transformation into Byte-wise features

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(X_res['protocol_m'])
X_res['protocol']=label

#X_res.drop("protocol_m", axis=1, inplace=True)
#X_res.drop("protocol_s", axis=1, inplace=True)
#X_res.drop("frame_num", axis=1, inplace=True)

X_res['attack_cat']=y_res

In [11]:
X_tr,Ytrain =payload_to_bytes(X_res,1500)

In [12]:
X_res.iloc[:,:-1]

,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,payload,t_delta,stime,label,protocol
0,205.185.208.27,443,192.168.10.15,60233,tcp,57,2960,17030310186261c249c0ba72ce9dc971d07711c1c371b1...,0.000319,1499367000,BENIGN,0
1,13.107.4.50,80,192.168.10.15,49672,tcp,118,2960,f9806d4baa3b99af2b0426787219da99656887dd70d68e...,0.000117,1499256360,BENIGN,0
2,192.168.10.3,61378,192.168.10.1,53,udp,128,83,d6d7010000010000000000010361647306616461707476...,0.000250,1499105296,BENIGN,1
3,192.168.10.3,60342,192.168.10.1,53,udp,128,76,4ede0100000100000000000107737570706f7274076d6f...,0.000439,1499359560,BENIGN,1
4,192.168.10.16,137,192.168.10.255,137,udp,64,78,35c3011000010000000000002046454546464446454543...,0.000105,1499178240,BENIGN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1371643,172.16.0.1,57684,192.168.10.50,80,tcp,62,435,474554202f64762f76756c6e65726162696c6974696573...,0.160490,1499347440,Web Attack – XSS,0
1371644,172.16.0.1,59042,192.168.10.50,80,tcp,62,637,474554202f64762f76756c6e65726162696c6974696573...,0.000003,1499347560,Web Attack – XSS,0
1371645,172.16.0.1,59042,192.168.10.50,80,tcp,62,435,474554202f64762f76756c6e65726162696c6974696573...,0.235294,1499347560,Web Attack – XSS,0
1371646,172.16.0.1,56994,192.168.10.50,80,tcp,62,637,474554202f64762f76756c6e65726162696c6974696573...,0.030888,1499347440,Web Attack – XSS,0


In [13]:
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,0])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,1])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,2])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,3])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,4])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,5])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,6])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,8])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,9])))

In [14]:
name=[]
for x in range(1,1501):
    name.append("payload_byte_"+str(x))
name.append("srcip")
name.append("srcport")
name.append("dstip")
name.append("dstport")
name.append("protocol_m")
name.append("ttl")
name.append("total_len")
name.append("t_delta")
name.append("Timestamp")

In [15]:
df = pd.DataFrame(X_tr, columns=name)
df['label']=Ytrain

In [16]:
df.iloc[:,:-1]

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1500,srcip,srcport,dstip,dstport,protocol_m,ttl,total_len,t_delta,Timestamp
0,23,3,3,16,24,98,97,194,73,192,...,172,205.185.208.27,443,192.168.10.15,60233,tcp,57,2960,0.000319,1499367000
1,249,128,109,75,170,59,153,175,43,4,...,184,13.107.4.50,80,192.168.10.15,49672,tcp,118,2960,0.000117,1499256360
2,214,215,1,0,0,1,0,0,0,0,...,0,192.168.10.3,61378,192.168.10.1,53,udp,128,83,0.00025,1499105296
3,78,222,1,0,0,1,0,0,0,0,...,0,192.168.10.3,60342,192.168.10.1,53,udp,128,76,0.000439,1499359560
4,53,195,1,16,0,1,0,0,0,0,...,0,192.168.10.16,137,192.168.10.255,137,udp,64,78,0.000105,1499178240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371643,71,69,84,32,47,100,118,47,118,117,...,0,172.16.0.1,57684,192.168.10.50,80,tcp,62,435,0.16049,1499347440
1371644,71,69,84,32,47,100,118,47,118,117,...,0,172.16.0.1,59042,192.168.10.50,80,tcp,62,637,0.000003,1499347560
1371645,71,69,84,32,47,100,118,47,118,117,...,0,172.16.0.1,59042,192.168.10.50,80,tcp,62,435,0.235294,1499347560
1371646,71,69,84,32,47,100,118,47,118,117,...,0,172.16.0.1,56994,192.168.10.50,80,tcp,62,637,0.030888,1499347440


In [18]:
df.to_csv(r'./CICIDS_results\Converted_data_CICIDS_final.csv',index=False)

In [ ]:
## Miniflow Files

In [5]:
aa = pd.read_csv('./CICIDS_results\Converted_data_CICIDS_final.csv')

In [21]:
import time
start_time = time.time()

grouped_df = aa.groupby(['srcip','srcport','dstip','dstport','protocol_m'], as_index=False)
i=0
for key, item in grouped_df:
    item.to_csv(f'./CICIDS_results/miniflow\miniflow-{i}.csv', index=False)
    i +=1

end_time = time.time()
print(f"elapsed miniflow time: {end_time - start_time} seconds.")

In [ ]:
## Packet File Generation

In [3]:
# df = pd.read_csv('./CICIDS_results/miniflow/miniflow-0.csv')

In [4]:
# print(df.head)

<bound method NDFrame.head of    payload_byte_1  payload_byte_2  payload_byte_3  payload_byte_4  \
0              67             194             216             240   
1             111             246              23               4   

   payload_byte_5  payload_byte_6  payload_byte_7  payload_byte_8  \
0             181             111             186             187   
1              33             158             156             127   

   payload_byte_9  payload_byte_10  ...        srcip  srcport          dstip  \
0             238              109  ...  1.226.51.14       80  192.168.10.12   
1             236              226  ...  1.226.51.14       80  192.168.10.12   

   dstport  protocol_m  ttl  total_len   t_delta   Timestamp   label  
0    55896         tcp   48       1500  0.000001  1499186460  BENIGN  
1    55896         tcp   48       1500  0.000000  1499186460  BENIGN  

[2 rows x 1510 columns]>


In [11]:
import os
import pandas as pd

start_algo1_time = time.time()

folder_path = r'./CICIDS_results\miniflow' # Replace with the path to your folder
output_path = r'./CICIDS_results\packets-7' # Replace with the path to your output folder
i = 1

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        if len(df) >= 7:
            output_filename = 'packet-{}.csv'.format(i)
            output_file_path = os.path.join(output_path, output_filename)
            df = df.sort_values(by=['Timestamp'])
            selected_df = df.head(7)
            selected_df.to_csv(output_file_path, index=False)
            i += 1

KeyboardInterrupt: 

In [ ]:
## Converting Packets to Flows

In [3]:
import pandas as pd
import glob

dfs = []
for file in glob.glob('./CICIDS_results/packets-7/*.csv'):
    df = pd.read_csv(file)
    dfs.append(df)

all_df = pd.concat(dfs, ignore_index=True)

In [4]:
all_df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,srcip,srcport,dstip,dstport,protocol_m,ttl,total_len,t_delta,Timestamp,label
0,22,3,1,0,144,1,0,0,140,3,...,192.168.10.25,51279,172.217.10.98,443,tcp,64,201,0.000826,1499094503,BENIGN
1,21,3,1,0,32,202,170,206,228,204,...,192.168.10.25,51279,172.217.10.98,443,tcp,64,89,0.000315,1499094743,BENIGN
2,21,3,1,0,32,202,170,206,228,204,...,192.168.10.25,51279,172.217.10.98,443,tcp,64,89,0.022976,1499094743,BENIGN
3,21,3,1,0,32,202,170,206,228,204,...,192.168.10.25,51279,172.217.10.98,443,tcp,64,89,0.065401,1499094872,BENIGN
4,21,3,1,0,32,202,170,206,228,204,...,192.168.10.25,51279,172.217.10.98,443,tcp,64,89,0.000315,1499094872,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248859,71,69,84,32,47,63,74,50,88,104,...,172.16.0.1,44934,192.168.10.50,80,tcp,62,482,0.000002,1499262420,DoS Hulk
248860,71,69,84,32,47,63,80,72,84,61,...,172.16.0.1,44934,192.168.10.50,80,tcp,62,382,0.000004,1499262540,DoS Hulk
248861,71,69,84,32,47,63,76,70,66,65,...,172.16.0.1,44934,192.168.10.50,80,tcp,62,450,0.000002,1499262600,DoS Hulk
248862,71,69,84,32,47,63,72,73,77,67,...,172.16.0.1,44934,192.168.10.50,80,tcp,62,377,0.000509,1499262600,DoS Hulk


In [5]:
print(df[['srcip', 'srcport', 'dstip', 'dstport']].isnull().sum())

srcip      0
srcport    0
dstip      0
dstport    0
dtype: int64


In [6]:
unique_protocols = all_df['protocol_m'].unique()

In [7]:
unique_protocols

array(['tcp', 'udp'], dtype=object)

In [8]:
protocol_map = {'tcp': 6, 'udp': 17, 'icmp': 1}
all_df['protocol_m'] = all_df['protocol_m'].map(protocol_map)

In [9]:
def create_flow_id(row):
    if row['label'] == 'BENIGN':
        return row['dstip'] + '-' + row['srcip'] + '-' + str(row['dstport']) + '-' + str(row['srcport']) + '-' + str(row['protocol_m'])
    else:
        return row['srcip'] + '-' + row['dstip'] + '-' + str(row['srcport']) + '-' + str(row['dstport']) + '-' + str(row['protocol_m'])
        
all_df['flow_id'] = all_df.apply(create_flow_id, axis=1)

In [10]:
# 53 more unique flows than Aden had.
all_df['flow_id'].nunique()

35552

In [11]:
all_df['flow_id']

0         172.217.10.98-192.168.10.25-443-51279-6
1         172.217.10.98-192.168.10.25-443-51279-6
2         172.217.10.98-192.168.10.25-443-51279-6
3         172.217.10.98-192.168.10.25-443-51279-6
4         172.217.10.98-192.168.10.25-443-51279-6
                           ...                   
248859        172.16.0.1-192.168.10.50-44934-80-6
248860        172.16.0.1-192.168.10.50-44934-80-6
248861        172.16.0.1-192.168.10.50-44934-80-6
248862        172.16.0.1-192.168.10.50-44934-80-6
248863        172.16.0.1-192.168.10.50-44934-80-6
Name: flow_id, Length: 248864, dtype: object

In [12]:
import pandas as pd

# create a list of unique flow IDs
flow_ids = all_df['flow_id'].unique()

# create a dataframe with columns for flow number and flow ID
df = pd.DataFrame({'flow_number': range(1, len(flow_ids)+1),
                   'flow_id': flow_ids})

# save the dataframe to a CSV file
df.to_csv('CICIDS-7-number-packets-flow_ids.csv', index=False)


In [13]:
import pandas as pd

# group packets by their flow_id
groups = all_df.groupby('flow_id')

In [14]:
# initialize an empty dataframe to store the extracted flow information
new_df = pd.DataFrame()

In [15]:
import pandas as pd

# define a function to extract flow information
def extract_flow_info(group):
    # take the mean of all three packet feature values for this flow
    flow_features = group.drop(['flow_id','srcip', 'srcport', 'dstip', 'dstport', 'Timestamp', 'label','protocol_m'], axis=1).mean(numeric_only=True)
    
    # extract the protocol of the flow
    flow_protocol = group['protocol_m'].unique()[0]
    
    # extract the label of the flow
    flow_label = group['label'].unique()[0]
    
    # return a new row with the mean features, the flow protocol, and the flow label
    return pd.concat([flow_features, pd.Series(flow_protocol), pd.Series(flow_label)])

# group packets by their flow_id and apply the extract_flow_info function to each group
new_df = all_df.groupby('flow_id').apply(extract_flow_info).reset_index(drop=True)
print("Grouped packets by flow_id and extracted flow information...")
# rename the protocol and label columns
#new_df = new_df.rename(columns={0: 'protocol_m', 1: 'label'})

# save the new dataframe to a new csv file
new_df.to_csv('CICIDS-7-number-packets-to-flows.csv', index=False)
print("Saved the new dataframe to csv file...")

end_algo1_time = time.time()
print(f"Elapsed Algorithm 1 time: {end_algo1_time - start_algo1_time} seconds.")

Grouped packets by flow_id and extracted flow information...
Saved the new dataframe to csv file...


In [28]:
import pandas as pd

df = pd.read_csv('./CICIDS-7-number-packets-to-flows_NEW.csv')
print(df.head())


   payload_byte_1  payload_byte_2  payload_byte_3  payload_byte_4  \
0       35.000000             2.0        9.285714           232.0   
1       35.000000             3.0        8.000000           233.0   
2       62.428571             2.0        7.285714           234.0   
3       23.000000             3.0        3.000000             0.0   
4       23.000000             3.0        3.000000             0.0   

   payload_byte_5  payload_byte_6  payload_byte_7  payload_byte_8  \
0        0.000000             0.0        9.000000      162.428571   
1        0.000000             0.0        9.428571      122.714286   
2        0.000000             0.0        7.000000      197.714286   
3      114.428571             0.0        0.000000        0.000000   
4       47.000000             0.0        0.000000        0.000000   

   payload_byte_9  payload_byte_10  ...  payload_byte_1496  payload_byte_1497  \
0             0.0              0.0  ...                0.0                0.0   
1       

In [29]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,35.000000,2.000000,9.285714,232.000000,0.000000,0.0,9.000000,162.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.116827,17,BENIGN
1,35.000000,3.000000,8.000000,233.000000,0.000000,0.0,9.428571,122.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.039666,17,BENIGN
2,62.428571,2.000000,7.285714,234.000000,0.000000,0.0,7.000000,197.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.205035,17,BENIGN
3,23.000000,3.000000,3.000000,0.000000,114.428571,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,128.0,159.428571,0.004582,6,BENIGN
4,23.000000,3.000000,3.000000,0.000000,47.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,92.000000,0.000152,6,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35547,89.857143,1.714286,7.714286,234.000000,0.000000,0.0,8.142857,163.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.062824,17,BENIGN
35548,62.428571,2.000000,6.714286,234.000000,0.000000,0.0,17.000000,93.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.096195,17,BENIGN
35549,35.000000,2.000000,7.714286,233.000000,0.000000,0.0,12.000000,108.857143,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.631200,17,BENIGN
35550,144.714286,2.000000,7.142857,233.857143,0.000000,0.0,5.000000,171.571429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.023520,17,BENIGN


In [30]:
# rename the column 'A' to 'X'
df = df.rename(columns={'0': 'protocol_m','0.1': 'label'})

In [31]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,35.000000,2.000000,9.285714,232.000000,0.000000,0.0,9.000000,162.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.116827,17,BENIGN
1,35.000000,3.000000,8.000000,233.000000,0.000000,0.0,9.428571,122.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.039666,17,BENIGN
2,62.428571,2.000000,7.285714,234.000000,0.000000,0.0,7.000000,197.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.205035,17,BENIGN
3,23.000000,3.000000,3.000000,0.000000,114.428571,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,128.0,159.428571,0.004582,6,BENIGN
4,23.000000,3.000000,3.000000,0.000000,47.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,92.000000,0.000152,6,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35547,89.857143,1.714286,7.714286,234.000000,0.000000,0.0,8.142857,163.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.062824,17,BENIGN
35548,62.428571,2.000000,6.714286,234.000000,0.000000,0.0,17.000000,93.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.096195,17,BENIGN
35549,35.000000,2.000000,7.714286,233.000000,0.000000,0.0,12.000000,108.857143,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.631200,17,BENIGN
35550,144.714286,2.000000,7.142857,233.857143,0.000000,0.0,5.000000,171.571429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.023520,17,BENIGN


In [32]:
# save the new dataframe to a new csv file
df.to_csv('CICIDS-7-number-packets-to-flows.csv', index=False)


In [33]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,35.000000,2.000000,9.285714,232.000000,0.000000,0.0,9.000000,162.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.116827,17,BENIGN
1,35.000000,3.000000,8.000000,233.000000,0.000000,0.0,9.428571,122.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.039666,17,BENIGN
2,62.428571,2.000000,7.285714,234.000000,0.000000,0.0,7.000000,197.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.205035,17,BENIGN
3,23.000000,3.000000,3.000000,0.000000,114.428571,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,128.0,159.428571,0.004582,6,BENIGN
4,23.000000,3.000000,3.000000,0.000000,47.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,92.000000,0.000152,6,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35547,89.857143,1.714286,7.714286,234.000000,0.000000,0.0,8.142857,163.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.062824,17,BENIGN
35548,62.428571,2.000000,6.714286,234.000000,0.000000,0.0,17.000000,93.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.096195,17,BENIGN
35549,35.000000,2.000000,7.714286,233.000000,0.000000,0.0,12.000000,108.857143,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.631200,17,BENIGN
35550,144.714286,2.000000,7.142857,233.857143,0.000000,0.0,5.000000,171.571429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.023520,17,BENIGN


In [34]:
df.label.value_counts()

DDoS                9561
DoS Hulk            6003
BENIGN              4967
DoS Slowhttptest    4215
FTP-Patator         3901
DoS slowloris       3872
SSH-Patator         2953
Bot                   44
PortScan              31
Infiltration           4
Heartbleed             1
Name: label, dtype: int64

In [35]:
labels = {'BENIGN','DDoS','DoS GoldenEye','DoS Hulk', 'FTP-Patator','SSH-Patator'}
test_df = df[df['label'].isin(labels)]

In [36]:
test_df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,35.000000,2.000000,9.285714,232.000000,0.000000,0.0,9.000000,162.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.116827,17,BENIGN
1,35.000000,3.000000,8.000000,233.000000,0.000000,0.0,9.428571,122.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.039666,17,BENIGN
2,62.428571,2.000000,7.285714,234.000000,0.000000,0.0,7.000000,197.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.205035,17,BENIGN
3,23.000000,3.000000,3.000000,0.000000,114.428571,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,128.0,159.428571,0.004582,6,BENIGN
4,23.000000,3.000000,3.000000,0.000000,47.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,92.000000,0.000152,6,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35547,89.857143,1.714286,7.714286,234.000000,0.000000,0.0,8.142857,163.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.062824,17,BENIGN
35548,62.428571,2.000000,6.714286,234.000000,0.000000,0.0,17.000000,93.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.096195,17,BENIGN
35549,35.000000,2.000000,7.714286,233.000000,0.000000,0.0,12.000000,108.857143,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.631200,17,BENIGN
35550,144.714286,2.000000,7.142857,233.857143,0.000000,0.0,5.000000,171.571429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.023520,17,BENIGN


In [37]:
test_df.label.value_counts()

DDoS           9561
DoS Hulk       6003
BENIGN         4967
FTP-Patator    3901
SSH-Patator    2953
Name: label, dtype: int64

In [3]:
import pandas as pd
df_p=pd.read_csv('./CICIDS2017_preprocessed.csv')

In [4]:
df_p

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,4.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,1.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.14-8.253.185.121-49486-80-6,8.253.185.121,80.0,192.168.10.14,49486.0,6.0,03/07/2017 08:56:22,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.14-8.253.185.121-49486-80-6,8.253.185.121,80.0,192.168.10.14,49486.0,6.0,03/07/2017 08:56:22,1.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.9-88-1031-6,192.168.10.9,1031.0,192.168.10.3,88.0,6.0,03/07/2017 08:56:38,609.0,7.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827672,192.168.10.15-72.21.91.29-61374-80-6,72.21.91.29,80.0,192.168.10.15,61374.0,6.0,7/7/2017 5:02,61.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2827673,192.168.10.15-72.21.91.29-61378-80-6,72.21.91.29,80.0,192.168.10.15,61378.0,6.0,7/7/2017 5:02,72.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2827674,192.168.10.15-72.21.91.29-61375-80-6,72.21.91.29,80.0,192.168.10.15,61375.0,6.0,7/7/2017 5:02,75.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2827675,192.168.10.15-8.41.222.187-61323-80-6,8.41.222.187,80.0,192.168.10.15,61323.0,6.0,7/7/2017 5:02,48.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [5]:
df_p.columns

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [6]:
df_p[df_p['Flow ID']=='192.168.10.3-192.168.10.9-88-1034-6']

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
7,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.9,1034.0,192.168.10.3,88.0,6.0,03/07/2017 08:56:38,524.0,7.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
8,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.3,88.0,192.168.10.9,1034.0,6.0,03/07/2017 08:56:38,6.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
426633,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.3,88.0,192.168.10.9,1034.0,6.0,03/07/2017 08:56:38,2.0,3.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
529515,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.9,1034.0,192.168.10.3,88.0,6.0,4/7/2017 8:57,534.0,7.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
529516,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.3,88.0,192.168.10.9,1034.0,6.0,4/7/2017 8:57,6.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
885286,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.3,88.0,192.168.10.9,1034.0,6.0,4/7/2017 8:57,2.0,3.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1955217,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.9,1034.0,192.168.10.3,88.0,6.0,6/7/2017 9:01,910.0,9.0,6.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1955276,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.9,1034.0,192.168.10.3,88.0,6.0,6/7/2017 9:01,97.0,1.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2124973,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.9,1034.0,192.168.10.3,88.0,6.0,7/7/2017 9:00,515.0,7.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2124974,192.168.10.3-192.168.10.9-88-1034-6,192.168.10.3,88.0,192.168.10.9,1034.0,6.0,7/7/2017 9:00,18.0,1.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [2]:
df = pd.read_csv('CICIDS-7-number-packets-to-flows_NEW.csv')

In [3]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,35.000000,2.000000,9.285714,232.000000,0.000000,0.0,9.000000,162.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.116827,17,BENIGN
1,35.000000,3.000000,8.000000,233.000000,0.000000,0.0,9.428571,122.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.039666,17,BENIGN
2,62.428571,2.000000,7.285714,234.000000,0.000000,0.0,7.000000,197.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.205035,17,BENIGN
3,23.000000,3.000000,3.000000,0.000000,114.428571,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,128.0,159.428571,0.004582,6,BENIGN
4,23.000000,3.000000,3.000000,0.000000,47.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,92.000000,0.000152,6,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35547,89.857143,1.714286,7.714286,234.000000,0.000000,0.0,8.142857,163.714286,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.062824,17,BENIGN
35548,62.428571,2.000000,6.714286,234.000000,0.000000,0.0,17.000000,93.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.096195,17,BENIGN
35549,35.000000,2.000000,7.714286,233.000000,0.000000,0.0,12.000000,108.857143,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.631200,17,BENIGN
35550,144.714286,2.000000,7.142857,233.857143,0.000000,0.0,5.000000,171.571429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,64.0,76.000000,0.023520,17,BENIGN


In [ ]:
def create_flow_id(row):
    if row['label'] == 'BENIGN':
        return row['dstip'] + '-' + row['srcip'] + '-' + str(row['dstport']) + '-' + str(row['srcport']) + '-' + str(row['protocol_m'])
    else:
        return row['srcip'] + '-' + row['dstip'] + '-' + str(row['srcport']) + '-' + str(row['dstport']) + '-' + str(row['protocol_m'])
        
df['flow_id'] = df.apply(create_flow_id, axis=1)

In [25]:
df.label.value_counts()

DDoS                9561
DoS Hulk            6003
BENIGN              4967
DoS Slowhttptest    4215
FTP-Patator         3901
DoS slowloris       3872
SSH-Patator         2953
Bot                   44
PortScan              31
Infiltration           4
Heartbleed             1
Name: label, dtype: int64

In [26]:
labels = {'BENIGN', 'DDoS', 'DoS GoldenEye', 'DoS Hulk', 'FTP-Patator', 'SSH-Patator'}
test_df = df[df['label'].isin(labels)]

In [27]:
test_df.label.value_counts()


DDoS           9561
DoS Hulk       6003
BENIGN         4967
FTP-Patator    3901
SSH-Patator    2953
Name: label, dtype: int64